<a href="https://colab.research.google.com/github/DhruvArora96/nfl_kaggle_competition-/blob/main/nfl_compeition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 3.3MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.10-cp36-none-any.whl size=73269 sha256=2df88ecaff58edbc95cb08385ba870b9f356225c32c6fd3d41b47109d1b55b7d
  Stored in directory: /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dhruvarora96","key":"193f6f8453f255dfe63703edb500f099"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!mkdir kaggle_data

mkdir: cannot create directory ‘kaggle_data’: File exists


In [ ]:
!kaggle competitions download -c nfl-impact-detection -p kaggle_data

In [ ]:
!unzip kaggle_data/nfl-impact-detection.zip

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!rsync --progress gdrive/MyDrive/Kaggle_outputs/train_images.zip train_images.zip

In [ ]:
!unzip train_images.zip -d train_images

In [12]:
!cp -r train_images/gdrive/MyDrive/Kaggle_outputs/train_images/ train_images

In [14]:
!mv train_images/train_images/ training_images

In [15]:
!rm -rf train_images

In [16]:
!mv training_images train_images

In [ ]:
!pip install -U torch torchvision

In [19]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import seaborn as sns
import cv2
from tqdm import tqdm
import torch
import torch.utils.data
from torchvision import transforms, utils

In [20]:
from torch.utils.data import DataLoader

In [21]:
train_df = pd.read_csv('train_labels.csv')
train_df = train_df.fillna(0)

In [22]:
image_names =[]
for i in tqdm(range(len(train_df))):
    image_names.append(train_df.video[i].replace('.mp4','_frame')+str(train_df.frame[i])+'.jpg')
train_df['imageNames'] = image_names

100%|██████████| 983885/983885 [00:19<00:00, 49635.74it/s]


In [23]:
frame_array = [-3,-2,-1,0,1,2,3]
impact_only_training_df = train_df[train_df['impact']==1]
for row in tqdm(impact_only_training_df[['video','frame']].values):
    train_df.loc[(train_df['video']==row[0])
                & (train_df['frame'].isin(row[1]+np.array(frame_array))),
                'impact']=1

100%|██████████| 2291/2291 [02:44<00:00, 13.90it/s]


In [24]:
video_path = 'train/'
image_path = 'train_images/'
#!mkdir train_images

In [25]:
images_list = train_df[train_df['impact']==1]
images_list=images_list[['video','frame']].drop_duplicates()
images_list=images_list.reset_index()
images_list=images_list.drop('index',axis=1)

In [26]:
def make_images(image_path,video_path,video_name,frame):
    vidcap = cv2.VideoCapture(video_path+video_name)
    success,_ = vidcap.read() 
    if success:
        vidcap.set(1,frame)
        _,image = vidcap.read()
        image_name = str(video_name).replace('.mp4','_frame')+str(frame)+'.jpg'
        cv2.imwrite(image_path+f'{image_name}',image)

In [ ]:
for i in tqdm(range(len(images_list))):
    make_images(image_path,video_path,images_list['video'][i],images_list['frame'][i])

100%|██████████| 8304/8304 [1:28:53<00:00,  1.56it/s]


In [27]:
image_names =[]
for i in tqdm(range(len(images_list))):
    image_names.append(images_list.video[i].replace('.mp4','_frame')+str(images_list.frame[i])+'.jpg')
images_list['imageNames'] = image_names

100%|██████████| 8304/8304 [00:00<00:00, 45153.37it/s]


In [28]:
train_df['x0']=train_df['left']
train_df['y0']=train_df['top']
train_df['x1']=train_df['left']+train_df['width']
train_df['y1']=train_df['top']+train_df['height']

In [103]:
class NFLDataset(torch.utils.data.Dataset):
    def __init__(self, ids, image_path,transform=None):
        self.transforms = transforms
        self.idx = ids
    def __getitem__(self,idx):
        target ={}
        image_name = images_list['imageNames'][idx]
        image=cv2.imread(image_path+str(image_name))
        boxes = train_df.loc[(train_df['imageNames']==image_name),['x0','y0','x1','y1']].values
        boxes=torch.as_tensor(boxes, dtype=torch.float32)
        target['boxes']=boxes

        return image,target
    def __len__(self):
        return len(self.idx)

In [104]:
NFL_dataset = NFLDataset(ids= list(images_list.index),image_path='train_images/')

In [105]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [106]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [107]:
data_loader = torch.utils.data.DataLoader(
        NFL_dataset,
        batch_size=4,
        pin_memory=False,
        drop_last=True,
        num_workers=2,
        collate_fn=collate_fn,
    )

In [112]:
images,targets=next(iter(data_loader))

In [113]:
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]

In [114]:
output = model(images,targets)

AttributeError: ignored